In [1]:
import googlemaps
import pandas as pd
import re
import numpy as np

In [4]:
import os
gmaps = googlemaps.Client(key=os.environ.get('GOOGLEMAPS_API_KEY'))

In [5]:
dtype = {year: np.int64 for year in range(2001,2020)}
df = pd.read_csv('data/census/1710013501-eng.csv', thousands=",", dtype=dtype)
df.head()

,Geography,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,All census metropolitan areas and census agglo...,25121190,25464948,25745344,26035077,26332444,26666157,26972594,27313943,27682785,...,28365332,28733706,29086709,29425513,29680350,30059970,30482398,30979142,31485410,31869048
1,"All census metropolitan areas, Canada",21174905,21500787,21753380,22011568,22277962,22584463,22857870,23163156,23502329,...,24124407,24454805,24776585,25084157,25315512,25660482,26044849,26495277,26959203,27306305
2,"St. John's (CMA), Newfoundland and Labrador",176468,177626,179026,181059,181720,183847,186173,189575,193867,...,202376,204537,206257,208228,209191,211003,212144,212344,213204,214014
3,"Halifax (CMA), Nova Scotia",369252,374445,378091,380752,381940,384658,386402,389919,393688,...,402074,404376,405598,407061,408017,414015,421713,429895,439529,448544
4,"Moncton (CMA), New Brunswick",123093,124560,126103,127702,128823,130507,132060,133860,136211,...,140698,142871,144217,145708,146367,148467,150438,152560,155870,158695


In [6]:
# remove total population counts and population counts of "rural area"
not_cities = df[df["Geography"].str.contains(r"[Aa]rea(s?) outside", regex=True) | df["Geography"].str.contains(r"metropolitan", regex=True)]
cities = df.drop(index=not_cities.index)

c:\users\tom\.virtualenvs\canada-population-map-n48wtm4g\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [7]:
# remove "(CA)" and "(CMA)" for city name
cities['city_name'] = cities['Geography'].map(lambda x: re.sub(r' \(CM?A\)', '', x))

In [8]:
def geocode(city):
    ''' Calls googlemaps geocode API using the formatted city_name row, 
        returns lat or lng from API call, as specified by direction'''
    results = gmaps.geocode(city)
    if results:
        return (results[0]['geometry']['location']['lat'], results[0]['geometry']['location']['lng'])
    else:
        print('No results for city: ', city)
        return (None, None)

In [9]:
cities['lat'], cities['lon'] = zip(*cities['city_name'].map(lambda x: geocode(x)))

No results for city:  Campbellton, New Brunswick/Quebec
No results for city:  Hawkesbury, Ontario/Quebec


In [10]:
# drop the two cities above
cities.drop(inplace=True, index=[54, 79])

In [11]:
for year in range(2002, 2020): 
    cities['diff_' + str(year)] = cities[str(year)] - cities[str(year-1)] 

In [12]:
cities.head()

,Geography,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,diff_2010,diff_2011,diff_2012,diff_2013,diff_2014,diff_2015,diff_2016,diff_2017,diff_2018,diff_2019
2,"St. John's (CMA), Newfoundland and Labrador",176468,177626,179026,181059,181720,183847,186173,189575,193867,...,4686,3823,2161,1720,1971,963,1812,1141,200,860
3,"Halifax (CMA), Nova Scotia",369252,374445,378091,380752,381940,384658,386402,389919,393688,...,4668,3718,2302,1222,1463,956,5998,7698,8182,9634
4,"Moncton (CMA), New Brunswick",123093,124560,126103,127702,128823,130507,132060,133860,136211,...,2214,2273,2173,1346,1491,659,2100,1971,2122,3310
5,"Saint John (CMA), New Brunswick",127363,127031,126858,126702,126186,126151,126700,127554,128691,...,734,502,-172,-559,-653,-261,644,557,767,800
6,"Saguenay (CMA), Quebec",162956,161506,160394,159504,158483,158043,158111,158707,158938,...,698,654,878,284,-65,-6,69,244,370,487


In [189]:
def get_color(row, year):
    diff = int(row['diff_'  + str(year)])
    if diff > 0:
        return "[0, 255, 0]" # growth
    if diff == 0: 
        return "[0, 0, 255]" # no change
    else: 
        return "[255, 0, 0]" # decline

for year in range(2002, 2020):
    cities['color_' + str(year)] = cities.apply(get_color, args=(year,), axis=1)

In [190]:
cities[["2002", "2003", "diff_2003", "color_2003"]]

,2002,2003,diff_2003,color_2003
2,177626,179026,1400,"[0, 255, 0]"
3,374445,378091,3646,"[0, 255, 0]"
4,124560,126103,1543,"[0, 255, 0]"
5,127031,126858,-173,"[255, 0, 0]"
6,161506,160394,-1112,"[255, 0, 0]"
...,...,...,...,...
158,87021,85664,-1357,"[255, 0, 0]"
159,11165,11205,40,"[0, 255, 0]"
160,24357,24709,352,"[0, 255, 0]"
161,22677,23180,503,"[0, 255, 0]"


In [13]:
cities.to_csv('data/cities.csv')